## Cheack observatoin data quality 

In [ ]:
from astropy.io import fits as pyfits
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import gt_apps as my_apps
from gt_apps import counts_map
import shutil
from os import listdir
from GtApp import GtApp
import urllib.request


In [ ]:
source_name = "J0001-1551"   # "J0001-1551"    # "J2158-3013"
data_path = "/Users/87steven/Documents/ASIAA/Blazar SED code and data/Fermi_all_sky_survey_data/"+source_name

RA = 0.272042  ### 0.272042    ### 329.716958
DEC = -15.851939    ### -15.851939    ### -30.225578
rad = 1
emin = 100
emax = 500000
energy_bin_num = 37
z_max = 90

### examine SCfile info. and observation data quality
sc_hdulist = pyfits.open(data_path+'/SC00.fits')

sc_hdulist.info()

sc_data = sc_hdulist[1].data

plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['axes.linewidth'] = 3
    
fig, ax = plt.subplots()

plt.plot(sc_data.field('START'), sc_data.field('DATA_QUAL'), '.')
plt.plot(sc_data.field('START'), sc_data.field('LAT_CONFIG'), '.')

ax.set_ylim(-0.1,1.1)

## Filter data

In [ ]:
%system ls "{data_path}"


In [ ]:
%system ls "{data_path}"/*PH* > "{data_path}"/events.txt


In [ ]:
%system cat "{data_path}"/events.txt


In [ ]:
%system gtvcut "{data_path}"/PH00.fits EVENTS


## gt_apps "filter" (gtselect)

In [ ]:
import gt_apps
from gt_apps import filter
filter.pars()

infile = " '" + data_path + "/events.txt' "
outfile = " '" + data_path + "/binned_filtered.fits' "

filter['infile'] = infile
filter['outfile'] = outfile
filter['ra'] = RA  ### seaching coordiante
filter['dec'] = DEC
filter['rad'] = rad   ### searching radius
filter['tmin'] = 'INDEF'
filter['tmax'] = 'INDEF'
filter['emin'] = emin    ### lower bound energy range
filter['emax'] = emax   ### higher bound energy range
filter['zmax'] = z_max   ### maximum zenith angle
filter['evclass'] = 128 ### 128 means "Galactic point source" & "off-plane point source" & "galactic diffuse source"
filter['evtype'] = 3


In [ ]:
### run the command
filter.run()


## Look filtered data

In [ ]:
filtered_data = pyfits.open(data_path+'/binned_filtered.fits')


In [ ]:
#filtered_data.info()


## gt_apps "maketime" (gtmktime)

will cost a lot of time

In [ ]:
from gt_apps import maketime

scfile = " '" + data_path + "/SC00.fits' "
evfile = " '" + data_path + "/binned_filtered.fits' "
outfile = data_path + "/binned_gti.fits "

maketime['evfile'] = evfile
maketime['scfile'] = scfile
maketime['outfile'] = outfile
maketime['filter'] = "DATA_QUAL > 0 && LAT_CONFIG == 1"
maketime['apply_filter'] = 'yes'
maketime['roicut'] = 'no'

maketime.run()


In [ ]:
outfile = data_path + "/binned_gti.fits"

mktime_data = pyfits.open(outfile)
mktime_data.info()


## Create a 3-D (binned) counts map

In [ ]:
scfile = " '" + data_path + "/SC00.fits' "
evfile = " '" + data_path + "/binned_gti.fits' "
outfile = " '" + data_path + "/binned_ccube.fits' "

### gt count maps
my_apps.counts_map['algorithm'] = 'CCUBE'
my_apps.counts_map['evfile'] = evfile
my_apps.counts_map['scfile'] = 'none'
my_apps.counts_map['outfile'] = outfile
my_apps.counts_map['nxpix'] = 100   #*rad
my_apps.counts_map['nypix'] = 100   #*rad
my_apps.counts_map['binsz'] = 0.2
my_apps.counts_map['coordsys'] = 'CEL'
my_apps.counts_map['xref'] = RA   # RA
my_apps.counts_map['yref'] = DEC    # DEC
my_apps.counts_map['axisrot'] = 0
my_apps.counts_map['proj'] = 'AIT'
my_apps.counts_map['ebinalg'] = 'LOG'
my_apps.counts_map['emin'] = emin
my_apps.counts_map['emax'] = emax
my_apps.counts_map['enumbins'] = energy_bin_num

my_apps.counts_map.run()


## Livetime Cube

cost time

In [ ]:
evfile = data_path + "/binned_gti.fits"
scfile = " '" + data_path + "/SC00.fits' "
outfile = " '" + data_path + "/ltcube.fits' "

### gt exp Cube
my_apps.expCube['evfile'] = evfile
my_apps.expCube['scfile'] = scfile
my_apps.expCube['outfile'] = outfile
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = rad
my_apps.expCube.run()


## Exposure Map

cost time

In [ ]:
### create model file
expCube2 = GtApp('gtexpcube2', 'Likelihood')

infile = " '" + data_path + "/ltcube.fits' "
cmap = 'none'
outfile = " '" + data_path + "/binned_exp_map.fits' "

expCube2['infile'] = infile
expCube2['cmap'] = cmap
expCube2['outfile'] = outfile
expCube2['irfs'] = 'P8R3_SOURCE_V3'
expCube2['evtype'] = '3'
expCube2['nxpix'] = 1800   #*rad
expCube2['nypix'] = 900    #*rad
expCube2['binsz'] = 0.2    #*rad
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = RA
expCube2['yref'] = DEC
expCube2['axisrot'] = 0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = emin
expCube2['emax'] = emax
expCube2['enumbins'] = energy_bin_num # bin number 
expCube2.run()



## Generate XML Model File


In [ ]:
### make4FGLxml.py
urllib.request.urlretrieve('https://fermi.gsfc.nasa.gov/ssc/data/analysis/user/make4FGLxml.py', 'make4FGLxml.py')

### gll_psc_v26.xml
urllib.request.urlretrieve('https://fermi.gsfc.nasa.gov/ssc/data/access/lat/10yr_catalog/gll_psc_v26.xml', data_path+'/gll_psc_v26.xml')

### iso_P8R3_SOURCE_V3_v1.txt
!ln -s $FERMI_DIR/refdata/fermi/galdiffuse/iso_P8R3_SOURCE_V3_v1.txt 
shutil.move("/Users/87steven/Documents/ASIAA/Blazar SED code and data/iso_P8R3_SOURCE_V3_v1.txt", data_path+"/iso_P8R3_SOURCE_V3_v1.txt")

### gll_iem_v07.fits
!ln -s $FERMI_DIR/refdata/fermi/galdiffuse/gll_iem_v07.fits 
shutil.move("/Users/87steven/Documents/ASIAA/Blazar SED code and data/gll_iem_v07.fits", data_path+"/gll_iem_v07.fits")


## Make model

In [ ]:
from make4FGLxml import *

mymodel = srcList(data_path+'/gll_psc_v26.xml', data_path+'/binned_gti.fits', data_path+'/model.xml')

mymodel.makeModel(data_path+'/gll_iem_v07.fits', 'gll_iem_v07.fits',  # .fits
                  data_path+'/iso_P8R3_SOURCE_V3_v1.txt', 'iso_P8R3_SOURCE_V3_v1')

### Interstellar model: gll_iem_v07.fits is from 4FGL DR1 paper
### instrunment response function: iso_P8R3_SOURCE_V3_v1

## Source Map

In [ ]:

expcube = " '" + data_path + "/ltcube.fits' "
cmap = " '" + data_path + "/binned_ccube.fits' "
srcmdl = " '" + data_path + "/model.xml' "
bexpmap = " '" + data_path + "/binned_exp_map.fits' "
outfile = " '" + data_path + "/binned_srcmaps.fits' "

my_apps.srcMaps['expcube'] = expcube
my_apps.srcMaps['cmap'] = cmap
my_apps.srcMaps['srcmdl'] = srcmdl
my_apps.srcMaps['bexpmap'] = bexpmap
my_apps.srcMaps['irfs'] = 'CALDB'
my_apps.srcMaps['outfile'] = outfile

my_apps.srcMaps.run()


## Compute the diffuse source responses.


In [ ]:
evfile = " '" + data_path + "/binned_gti.fits' "
scfile = " '" + data_path + "/SC00.fits' "
srcmdl = " '" + data_path + "/model.xml' "

my_apps.diffResps['evfile'] = evfile 
my_apps.diffResps['scfile'] = scfile 
my_apps.diffResps['srcmdl'] = srcmdl 
my_apps.diffResps['irfs'] = 'CALDB'
my_apps.diffResps.run()



## Run the Likelihood Analysis


In [ ]:
import pyLikelihood
from BinnedAnalysis import *

srcmap = data_path + "/binned_srcmaps.fits"
bexpmap = data_path + "/binned_exp_map.fits"
expcube = data_path + "/ltcube.fits"

obs = BinnedObs(srcMaps = srcmap, binnedExpMap = bexpmap, expCube = expcube, irfs = 'CALDB')

like = BinnedAnalysis(obs, data_path+'/model.xml', optimizer = 'NewMinuit')


In [ ]:
### perform model fit
likeobj = pyLikelihood.NewMinuit(like.logLike)

like.fit(verbosity = 0, covar = True, optObject = likeobj)


In [ ]:
### print out fitted parameters of each FGL source

#like.model

### can specify source name to print out fitted parameters
#like.model['4FGL J0025.2-2231']

### print out all source name
#name_list = like.sourceNames()


## Test Statistic (TS) value

TS > 25 is generally considered a strong detection with a high likelihood of being real.

In [ ]:
name_list = np.array(like.sourceNames())
Ts_array = np.array([ ])
 
for i in name_list:
    Ts = like.Ts(i)
    Ts_array = np.append(Ts_array, Ts)
    #print(i, '=', Ts)

Ts_array = Ts_array[0:(len(Ts_array))-2]
index = np.where(Ts_array == np.max(Ts_array))[0]
best_source = name_list[index][0]

print("Maximum Ts source name = ", best_source, ", Ts =", Ts_array[index][0])


In [ ]:
### Read original SED data
test = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/source individual flux 11_15/' + source_name + '_flux.csv') 

freq_ori = test.freq.values
flux_ori = test.flux.values
flux_err_ori = test.flux_err.values

### create energy bin
energy_bin = 10**np.linspace( np.log10(np.min(like.energies)), np.log10(np.max(like.energies)), 10)

fermi_freq = []
flux_array = []
flux_err_array = []

### transform counts into flux density
for i in range(0, len(energy_bin)-1):

    flux = like.flux(best_source,emin = energy_bin[i], emax = energy_bin[i+1], energyFlux = False)
    flux_array.append(flux)
    
    flux_err = like.fluxError(best_source, emin = energy_bin[i], emax = energy_bin[i+1])
    flux_err_array.append(flux_err)
    
    log_low_energy = np.log10(energy_bin[i])
    log_high_energy = np.log10(energy_bin[i+1]) 
    log_energy_diff = log_high_energy-log_low_energy 
    log_energy = log_low_energy+log_energy_diff
    
    fermi_freq.append(10**log_energy*1.0E6/(4.1356E-15))

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['axes.linewidth'] = 3
    
fig, ax = plt.subplots()

ax.errorbar(np.log10(freq_ori), np.log10(flux_ori), yerr = flux_err_ori, marker = "o", color = "red", ecolor = "red", linestyle = '', label = 'Total SED') 

#ax.plot(np.log10(fermi_freq), np.log10(flux_array), marker = ".", color = "blue", linestyle = '', label = 'Fermi')
ax.errorbar(np.log10(fermi_freq), np.log10(flux_array), yerr = 0.434*np.array(flux_err_array)/flux_array, marker = "o", color = "blue", ecolor = "blue", linestyle = '', label = 'Fermi') 

ax.set_xlim(7, 29)
ax.set_ylim(-16, -9) 

plt.legend(loc = 'upper left', shadow = True, fontsize = 20, markerscale = 1) 

#ax.set_xscale('log')
#ax.set_yscale('log')
plt.xlabel( "log($\\nu$) [Hz]",  fontsize = 16)
plt.xticks(fontsize = 14)
plt.ylabel('log($\\nu$ F($\\nu$)) [erg $\mathrm{cm^{-2}\ s^{-1}}$]',  fontsize = 16)
plt.yticks(fontsize = 14)
plt.grid(True)
figurename = source_name+' SED Test'
plt.title(figurename, fontsize = 20)


In [ ]:
#help(like)

## Plot figure

In [ ]:
import numpy as np

E = (like.energies[:-1] + like.energies[1:])/2.
name_list = list( like.sourceNames() )
count = list(like.nobs)

sum_model = np.zeros_like( like._srcCnts(like.sourceNames()[0]) )

plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['axes.linewidth'] = 3
    
fig, ax = plt.subplots()

for i in range(30, len(name_list)):
    sum_model = sum_model + like._srcCnts(name_list[i])
    plt.loglog(E, like._srcCnts(name_list[i]), label = name_list[i] )
    
plt.loglog(E, sum_model, '--', color = 'red', label = 'Total Model')
plt.errorbar(E, count, yerr = np.sqrt(like.nobs), color = 'blue', fmt = 'o', label = 'Counts')

plt.xlabel('Energy [MeV]',  fontsize = 15)
plt.ylabel('Counts/bin',  fontsize = 15)

#plt.xlim(200, 300000)
plt.ylim(0.1, 1e3)

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.5, box.height])

plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad=0., ncol = 2)

ax.tick_params(width = 2, direction = "in", length = 10, labelsize = 15)
ax.tick_params(which = 'minor', direction = "in", width = 2, length = 6, color = 'k')

ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')


In [ ]:
residual = (like.nobs - sum_model)/sum_model   
resid_err = np.sqrt(like.nobs)/sum_model

plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['axes.linewidth'] = 3
    
fig, ax = plt.subplots()

plt.plot(E, residual, 'o', color = 'red')
plt.errorbar(E, residual, yerr = resid_err, color = 'blue', fmt = 'o')

plt.xlabel('Energy [MeV]',  fontsize = 15)
plt.ylabel('Residual [(count-model)/model]',  fontsize = 15)

plt.xlim(200, 300000)
#plt.ylim(-1.01, 115)


#plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad=0., ncol = 2)

ax.set_xscale('log')

ax.tick_params(width = 2, direction = "in", length = 10, labelsize = 15)
ax.tick_params(which = 'minor', direction = "in", width = 2, length = 6, color = 'k')

ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')


In [ ]:

E = (like.energies[:-1] + like.energies[1:])/2.
freq = E*1.0E6/(4.1356E-15)
name_list = list( like.sourceNames() )
count = list(like.nobs)

plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['axes.linewidth'] = 3
    
fig, ax = plt.subplots()

for i in range(40, len(name_list)):
    plt.plot(np.log10(freq), like._srcCnts(name_list[i]), label = name_list[i] )
    
plt.plot(np.log10(freq), sum_model, '--', color = 'red', label = 'Total Model')
plt.errorbar(np.log10(freq), count, yerr = np.sqrt(like.nobs), color = 'blue', fmt = 'o', label = 'Counts')

plt.xlabel('log($\\nu$) [Hz]',  fontsize = 15)
plt.ylabel('Counts/bin',  fontsize = 15)

#plt.xlim(200, 300000)
ax.set_yscale('log')
plt.ylim(0.1, 1e3)

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.5, box.height])

plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad=0., ncol = 2)

ax.tick_params(width = 2, direction = "in", length = 10, labelsize = 15)
ax.tick_params(which = 'minor', direction = "in", width = 2, length = 6, color = 'k')

ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')


In [213]:
like.model['4FGL J0006.3-0620']


4FGL J0006.3-0620
   Spectrum: PowerLaw
14     Prefactor:  1.665e+00  0.000e+00  0.000e+00  1.000e+02 ( 1.998e-14) fixed
15         Index:  2.128e+00  0.000e+00  0.000e+00  5.000e+00 (-1.000e+00) fixed
16         Scale:  2.083e+00  0.000e+00  1.000e-02  1.000e+03 ( 1.000e+03) fixed

In [214]:
like.flux('4FGL J0006.3-0620',emin=100)

1.8872077680579995e-09

In [215]:
like.fluxError('4FGL J0006.3-0620')



0

In [ ]:
aa = %system exiftool '/Users/87steven/Documents/2023_1_13_鹿林天文台/2023_1_16 RAW/DSC09999.ARW'

aa